In [14]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, time

In [25]:
input_file_path = '/Users/kimd999/research/projects/Katrina/per_each_data/extracts/input/latest/ZF_120_SRP_Sample_Extracts_with_Dilution_Factors_2021MAY21.csv'
df = pd.read_csv(input_file_path, header = 0)
df.shape

(570239, 8)

In [3]:
df.head(2400).tail

<bound method NDFrame.tail of       chemical.id bottle.id   conc  plate.id well      date endpoint  value
0             101    C51699  0.000     10021  A06  20151113     MO24    0.0
1             101    C51699  0.000     10021  B06  20151113     MO24    0.0
2             101    C51699  0.000     10021  C06  20151113     MO24    0.0
3             101    C51699  0.000     10021  D06  20151113     MO24    1.0
4             101    C51699  0.000     10021  E06  20151113     MO24    0.0
...           ...       ...    ...       ...  ...       ...      ...    ...
2395          798    P84353  0.005      8118  D02  20150722     MO24    0.0
2396          798    P84353  0.005      8118  E02  20150722     MO24    0.0
2397          798    P84353  0.005      8118  F02  20150722     MO24    0.0
2398          798    P84353  0.005      8118  G02  20150722     MO24    0.0
2399          798    P84353  0.005      8118  H02  20150722     MO24    0.0

[2400 rows x 8 columns]>

In [9]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
df_select = df.loc[:,columns_to_keep]
df_select.head()

,chemical.id,conc,plate.id,well,endpoint,value
0,101,0.0,10021,A06,MO24,0.0
1,101,0.0,10021,B06,MO24,0.0
2,101,0.0,10021,C06,MO24,0.0
3,101,0.0,10021,D06,MO24,1.0
4,101,0.0,10021,E06,MO24,0.0


In [15]:
start_time = time.time()

df_reformatted = pd.DataFrame()
total_number_of_unique_chemicals = 0
total_number_of_chemical_plate_well = 0

chemical_id_from_here = np.unique(df['chemical.id'])

total_number_of_chemicals_to_processed = len(chemical_id_from_here)
number_of_chemicals_processed = 0

for chemical_index in chemical_id_from_here:
    print("\nchemical_index:" + str(chemical_index))        
    total_number_of_unique_chemicals += 1
#    if (chemical_index != 53):# and (chemical_index != 3859)):
 #       continue

    df_per_chemical = df_select.loc[df['chemical.id'] == chemical_index,:]
    #    print ("\ntotal_number_of_unique_chemicals:" + str(total_number_of_unique_chemicals))

    # Append chemical_plate_well as a unique identifier
    df_per_chemical.insert(0, 'chemical_plate_well', df_per_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    print ("len(df_per_chemical.chemical_plate_well):" + str(len(df_per_chemical.chemical_plate_well)))    

    for cpw in np.unique(df_per_chemical.chemical_plate_well):
        total_number_of_chemical_plate_well += 1
        temp_df = df_per_chemical.loc[df_per_chemical.chemical_plate_well == cpw,:]
        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        #print ("temp_df_grouped:" + str(temp_df_grouped)) #<pandas.core.groupby.generic.DataFrameGroupBy object at 0x1292f0190>

        for name, group in temp_df_grouped:
#            if(len(group.endpoint) == 23):
#                try:
                    temp = pd.DataFrame(
                        {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc']),
                        'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                        'BRAI': temp_df.value[temp_df.endpoint == 'BRAI'].values,
                        'CFIN': temp_df.value[temp_df.endpoint == 'CFIN'].values,
                        'CIRC': temp_df.value[temp_df.endpoint == 'CIRC'].values,
                        'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                        'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                        'EYE_': temp_df.value[temp_df.endpoint == 'EYE_'].values,
                        'JAW_': temp_df.value[temp_df.endpoint == 'JAW_'].values,
                        'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                        'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                        'NC24': temp_df.value[temp_df.endpoint == 'NC24'].values,
                        'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                        'OTIC': temp_df.value[temp_df.endpoint == 'OTIC'].values,
                        'PE__': temp_df.value[temp_df.endpoint == 'PE__'].values,
                        'PFIN': temp_df.value[temp_df.endpoint == 'PFIN'].values,
                        'PIG_': temp_df.value[temp_df.endpoint == 'PIG_'].values,
                        'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                        'SNOU': temp_df.value[temp_df.endpoint == 'SNOU'].values,
                        'SOMI': temp_df.value[temp_df.endpoint == 'SOMI'].values,
                        'SWIM': temp_df.value[temp_df.endpoint == 'SWIM'].values,
                        'TRUN': temp_df.value[temp_df.endpoint == 'TRUN'].values,
                        'TR__': temp_df.value[temp_df.endpoint == 'TR__'].values,
                        'YSE_': temp_df.value[temp_df.endpoint == 'YSE_'].values,
                        }
                            )
                    df_reformatted = pd.concat([df_reformatted, temp])
              #  except:
              #      print ("len(group.endpoint):" + str(len(group.endpoint)))
              #      print ("chemical_plate_well:" + str(cpw))
    number_of_chemicals_processed += 1
    print_this = str(number_of_chemicals_processed) + " chemicals processed out of " + str(total_number_of_chemicals_to_processed)
    print(print_this)
    
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    
print ("\ntotal_number_of_unique_chemicals:" + str(total_number_of_unique_chemicals))
print ("total_number_of_chemical_plate_well:" + str(total_number_of_chemical_plate_well))
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Done, it took:"+str(time_took)) 
# took 150 seconds for all 42 chemicals


chemical_index:101
len(df_per_chemical.chemical_plate_well):4416
1 chemicals processed out of 84
Current Time = 04:59:49

chemical_index:102
len(df_per_chemical.chemical_plate_well):4416
2 chemicals processed out of 84
Current Time = 04:59:51

chemical_index:103
len(df_per_chemical.chemical_plate_well):4416
3 chemicals processed out of 84
Current Time = 04:59:53

chemical_index:131
len(df_per_chemical.chemical_plate_well):4048
4 chemicals processed out of 84
Current Time = 04:59:54

chemical_index:132
len(df_per_chemical.chemical_plate_well):4048
5 chemicals processed out of 84
Current Time = 04:59:56

chemical_index:177
len(df_per_chemical.chemical_plate_well):4048
6 chemicals processed out of 84
Current Time = 04:59:57

chemical_index:452
len(df_per_chemical.chemical_plate_well):4048
7 chemicals processed out of 84
Current Time = 04:59:59

chemical_index:495
len(df_per_chemical.chemical_plate_well):4048
8 chemicals processed out of 84
Current Time = 05:00:01

chemical_index:669
len(

In [16]:
pd.set_option('display.max_columns', None)
print (df_reformatted.head())
print ("df_reformatted.shape:" + str(df_reformatted.shape))

   chemical.id  plate.id well chemical_plate_well    conc  AXIS  BRAI  CFIN  \
0          101     10021  A01       101_10021_A01  0.0100   NaN   NaN   NaN   
0          101     10021  A02       101_10021_A02  0.0050   0.0   0.0   0.0   
0          101     10021  A03       101_10021_A03  0.0025   0.0   0.0   0.0   
0          101     10021  A04       101_10021_A04  0.0010   0.0   0.0   0.0   
0          101     10021  A05       101_10021_A05  0.0001   0.0   0.0   0.0   

   CIRC  DNC_  DP24  EYE_  JAW_  MO24  MORT  NC24  NC__  OTIC  PE__  PFIN  \
0   NaN   0.0   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN   
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   PIG_  SM24  SNOU  SOMI  SWIM  TRUN  TR__  YSE_  
0   NaN   

In [18]:
df_reformatted_DNC_0 = pd.DataFrame()

In [19]:
df_reformatted_DNC_0 = df_reformatted.loc[df_reformatted['DNC_'] == 0.0]
print ("df_reformatted_DNC_0.shape:" + str(df_reformatted_DNC_0.shape))

df_reformatted_DNC_0.shape:(24768, 28)


In [26]:
output_file_path = input_file_path[:-4] + "_wide_DNC_0.csv"

print (f"output_file_path:\n{output_file_path}")

df_reformatted_DNC_0.to_csv(output_file_path,index=False)

output_file_path:
/Users/kimd999/research/projects/Katrina/per_each_data/extracts/input/latest/ZF_120_SRP_Sample_Extracts_with_Dilution_Factors_2021MAY21_wide_DNC_0.csv


In [ ]:
a=b

In [19]:
np.sum(df['value'] == 1)

7399

In [5]:
np.sum(df['value'] == 0) # -> 82%

472870

In [6]:
np.sum(np.isnan(df['value']))

68154

In [ ]:
'''
nan_count = 0
zero_count = 0
one_count = 0
for (columnName, columnData) in reformat_data.iteritems(): 
    if (columnName == "chemical.id") or (columnName == "plate.id") or (columnName == "well") or (columnName == "chemical_plate_well") or (columnName == "conc"):
        continue
#    print('Colunm Name : ', columnName) 
#    print('Column Contents : ', columnData.values[i])
#    print (len(columnData.values))
    for i in range(len(columnData.values)):
        if (str(columnData.values[i]) == "nan"):
            nan_count += 1
        elif (str(columnData.values[i]) == "0.0"):
            zero_count += 1
        elif (str(columnData.values[i]) == "1.0"):
            one_count += 1
#        print('Column Contents : ', columnData.values[i])
print ("nan_count:" + str(nan_count))
print ("zero_count:" + str(zero_count))
print ("one_count:" + str(one_count))
'''